In [1]:
import os
from glob import glob
import numpy as np
import dask
import xarray as xr
import cartopy.crs as ccrs
from cmocean import cm
import xgcm

from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = 12, 8

from mitequinox.utils import *

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/cmocean/tools.py:76: MatplotlibDeprecationWarning: The is_string_like function was deprecated in version 2.1.
  if not mpl.cbook.is_string_like(rgbin[0]):


In [2]:
from dask_jobqueue import PBSCluster
local_dir = os.getenv('TMPDIR')
cluster = PBSCluster(local_directory=local_dir)
#print(cluster.job_script())
w = cluster.start_workers(35)

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/distributed/utils.py:128: RuntimeWarning: Couldn't detect a suitable IP address for reaching '8.8.8.8', defaulting to '127.0.0.1': [Errno 101] Network is unreachable
  % (host, default, e), RuntimeWarning)


In [3]:
# get dask handles and check dask server status
from dask.distributed import Client
client = Client(cluster)

In [4]:
client

Client Scheduler: tcp://10.148.1.0:58052 Dashboard: http://10.148.1.0:8787/status,Cluster Workers: 70 Cores: 280 Memory: 3500.00 GB


_________
# Read dataset


In [5]:
## open files, nc files  ##
ncdata_dir = '/home1/scratch/xyu/mit_nc_new/'

# number of fies to load for each face (1 file = 10 days --- well, now it depends...)
Nf = 1
face_60 = [1]

v_u = 'SSU'
v_v = 'SSV'
v_eta = 'Eta'


In [ ]:
## Here we do DFT on u+iv ##

for face in face_60:
#for face in range(8,9):
    
    # SSU #
    filenames_u = sorted(glob(ncdata_dir+v_u+'_f%02d_t*.nc' %(face)))
    filenames_u = filenames_u[:Nf]
    
    # SSV #
    filenames_v = sorted(glob(ncdata_dir+v_v+'_f%02d_t*.nc' %(face)))
    filenames_v = filenames_v[:Nf]    

    # Eta#
    filenames_Eta = sorted(glob(ncdata_dir+v_eta+'_f%02d_t*.nc' %(face)))
    filenames_Eta = filenames_Eta[:Nf]        
    
    # grid file
    grid = xr.open_dataset('/home1/scratch/xyu/mit_grid_t.nc')
    grid = grid.isel(face=face_60)
    
    # Read SSU #
    ds = xr.open_mfdataset(filenames_u, chunks={'i_g':10, 'j':10}, concat_dim='time', compat='equals')
    chunks={'time': 1}
    ds = ds.chunk(chunks)    

    # Read SSV #
    ds_v = xr.open_mfdataset(filenames_v, chunks={'i':10, 'j_g':10}, concat_dim='time', compat='equals')
    chunks_v={'time': 1}
    ds_v = ds_v.chunk(chunks_v)
    
    # Read Eta #
    ds_Eta = xr.open_mfdataset(filenames_Eta, chunks={'i':10, 'j':10}, concat_dim='time', compat='equals')
    chunks_Eta={'time': 1}
    ds_Eta = ds_Eta.chunk(chunks_Eta)    
    
    # Merge into one xarray dataset #
    ds = xr.merge([ds_v,ds,ds_Eta,grid])
    print(ds)

In [ ]:
time_begin = 1
time_end = 119

---


# 1. Gradient of Bernoulli function (Eta)

In [ ]:
#import xgcm
grid = xgcm.Grid(ds, periodic=['X', 'Y'])
grid

# 1.2 Gradient of 'pressure' (i.e. dphi/dy)

In [ ]:
g = 9.81
dphi_dy = (grid.diff(g*ds['Eta'],'Y', boundary='extend'))/ds.dyC
dphi_dy_face2 = dphi_dy.isel(time=slice(time_begin,time_end),face=0)
print(dphi_dy_face2)


_______

# 2 Linear Coriolis terms

## 2.1 G_u(fv) ---> SSV

In [ ]:
SSU_dyG = ds.dyG * ds['SSU']
dyGSSU_i = grid.interp(SSU_dyG,'X')
dyGSSU_ij = grid.interp(dyGSSU_i,'Y')
print(dyGSSU_ij)


In [ ]:
lat = ds['YG']
omega = 7.3/100000
f = 2*omega*np.sin(np.deg2rad(lat))
f_i = grid.interp(f,'X')
Coriolis_v = -f_i * dyGSSU_ij /ds.dyC
Coriolis_v_face2 = Coriolis_v.isel(time=slice(time_begin,time_end))
print(Coriolis_v_face2)


# 3 Time derivative of SSV

In [ ]:
SSV_forward = ds['SSV'].isel(time=slice(time_begin+1,time_end+1))
SSV_forward = SSV_forward.drop(['dtime','time'])

SSV_backward = ds['SSV'].isel(time=slice(time_begin-1,time_end-1))
SSV_backward = SSV_backward.drop(['dtime','time'])

d_SSV_dt_center_face2 = (SSV_forward - SSV_backward)/2/3600
print(d_SSV_dt_center_face2)

# 4. Relative vorticity

In [ ]:
SSV_dyC = ds.dyC * ds['SSV']
d_SSV_dyC_di = grid.diff(SSV_dyC,'X')

SSU_dxC = ds.dxC * ds['SSU']
d_SSU_dxC_dj = grid.diff(SSU_dxC,'Y')

zeta_3 = (d_SSV_dyC_di - d_SSU_dxC_dj)/ds.rAz
zeta_3_i = grid.interp(zeta_3,'X')

Coriolis_zeta3_v = -zeta_3_i * dyGSSU_ij /ds.dyC
Coriolis_zeta3_v_face2 = Coriolis_zeta3_v.isel(time=slice(time_begin,time_end))

print(Coriolis_zeta3_v_face2)

# 5. KE

In [ ]:
SSU_2 = ds['SSU'] * ds['SSU']
SSU_2_i = grid.interp(SSU_2,'X')

SSV_2 = ds['SSV'] * ds['SSV']
SSV_2_j = grid.interp(SSV_2,'Y')

KE = (SSU_2_i + SSV_2_j)/2

dKE_dy = (grid.diff(KE,'Y', boundary='extend'))/ds.dyC
dKE_dy_face2 = dKE_dy.isel(time=slice(time_begin,time_end))
print(dKE_dy_face2)


# 6. Residuals

In [ ]:
residuals_v_face2 = d_SSV_dt_center_face2 - Coriolis_v_face2 - Coriolis_zeta3_v_face2 + dphi_dy_face2 + dKE_dy_face2
print(residuals_v_face2)

In [ ]:
fig = plt.figure(figsize = (24,12))

plt.subplot(231)
Coriolis_v_face2.isel(time=10).plot(vmin=-0.0001, vmax=0.0001, cmap = cm.balance)
plt.subplot(234)
dphi_dy_face2.isel(time=10).plot(vmin=-0.0001, vmax=0.0001, cmap = cm.balance)

plt.subplot(232)
Coriolis_zeta3_v_face2.isel(time=10).plot(vmin=-0.0001, vmax=0.0001, cmap = cm.balance)
plt.subplot(235)
dKE_dy_face2.isel(time=10).plot(vmin=-0.0001, vmax=0.0001, cmap = cm.balance)

plt.subplot(233)
d_SSV_dt_center_face2.isel(time=10).plot(vmin=-0.0001, vmax=0.0001, cmap = cm.balance)
plt.subplot(236)
residuals_v_face2.isel(time=10).plot(vmin=-0.0001, vmax=0.0001, cmap = cm.balance)


In [ ]:
residuals_v_face2_mean = residuals_v_face2.mean(['time']).persist()

In [ ]:
residuals_v_face2_mean.plot(vmin=-0.0001, vmax=0.0001, cmap = cm.balance)

In [ ]:
residuals_v_face2.std(['time']).plot(vmin=0, vmax=0.000025, cmap = cm.balance)

In [44]:
cluster.scheduler.close()

<Future finished result=None>